This notebook allows you to run the plenoctrees conversion of a jaxnerf checkpoint. It takes as input a JaxNeRF checkpoint file and outputs a .npz optmized plenoctree file, which can optionally be compressed into the same file format. This can then be used to be rendered on the given volrend or nerfvis methods.

Note that the same code can also be used for NeRF-SH inputs, you'll just have to remove the -s_jaxnerf argument in the python execution code for extraction.

# Input & Google Drive setup

In [ ]:
# If running on google colab, execute this to mount to your google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [1]:
import os
from glob import glob

# Synthetic scene to run. Choose from: [chair, drums, ficus, hotdog, lego, materials, mic, ship]
scene = 'drums'

############ Define ABSOLUTE paths ############

# Abolute path in which the plenoctrees code is present. This path should contain the requirements.txt file
path_code="/content/drive/MyDrive/plenoctree" 

# Change directory to where the code resides
%cd {path_code}
%ls

# Data folder
path_data=os.getcwd()+f'/data/NeRF_Data/nerf_synthetic/{scene}/'

# Folder to store checkpoints and other outputs
path_checkpoint=os.getcwd()+f"/data/pretrained/jaxnerf/{scene}/"

# Path to the configuration file. Note that this points to a .yaml file. 
# Input the entire path until the name of the yaml file.
# For example, if the file path is: /users/documents/config.yaml --> path_config = /users/documents/config
path_config=os.getcwd()+'/nerf_sh/config/misc/proj'

# Install dependencies

In [7]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.3/274.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [8]:
!pip show jax jaxlib flax optax svox gpustat

Name: jax
Version: 0.4.23
Summary: Differentiate, compile, and transform Numpy code.
Home-page: https://github.com/google/jax
Author: JAX team
Author-email: jax-dev@google.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: ml-dtypes, numpy, opt-einsum, scipy
Required-by: chex, dopamine-rl, flax, optax, orbax-checkpoint
---
Name: jaxlib
Version: 0.4.23+cuda12.cudnn89
Summary: XLA library for JAX
Home-page: https://github.com/google/jax
Author: JAX team
Author-email: jax-dev@google.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: ml-dtypes, numpy, scipy
Required-by: chex, dopamine-rl, optax, orbax-checkpoint
---
Name: flax
Version: 0.8.1
Summary: Flax: A neural network library for JAX designed for flexibility
Home-page: 
Author: 
Author-email: Flax team <flax-dev@google.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: jax, msgpack, numpy, optax, orbax-checkpoint, PyYAML, rich, tensorstore, typin

# Conversion of Vanilla JaxNeRF to PlenOctrees

In [ ]:
%%time

# Extract octrees from nerfsh. 
!python -m octree.extraction \
    --train_dir {path_checkpoint} --is_jaxnerf_ckpt \
    --config {path_config} \
    --data_dir {path_data} \
    --output {path_checkpoint}/octrees/tree.npz \
    --projection_samples 5000 \
    --radius 1.3

In [ ]:
%%time
# Optimize octrees
!python -m octree.optimization \
    --input {path_checkpoint}/octrees/tree.npz \
    --config {path_config} \
    --data_dir {path_data} \
    --output {path_checkpoint}/octrees/tree_opt.npz \
    --train_dir {path_checkpoint}

In [ ]:
%%time
# Evaluate octrees vs test set
#! add train dir to save images
!python -m octree.evaluation \
    --input {path_checkpoint}/octrees/tree_opt.npz \
    --config {path_config} \
    --data_dir {path_data} \
    --write_images {path_checkpoint}/octrees/tree_render \
    --write_vid {path_checkpoint}/octrees/tree_render/rendered.mp4

In [ ]:
%%time
# [Optional] Only used for in-browser viewing.
!python -m octree.compression \
    {path_checkpoint}/octrees/tree_opt.npz \
    --out_dir {path_checkpoint} \
    --overwrite